In [1]:
#importing dependencies
import      pandas as pd
import      sqlite3
import      csv
from        sqlite3 import Error

## DB Connection

In [2]:
def create_connection(path):
    '''
    This function creats a connection with sqlite
    '''
    connection =None
    try: 
        connection = sqlite3.connect(path)
        print("connection to SQLite DB succesful")
    except Error as e:
        print(f"The error '{e}' occured")
    return connection

#When executing a query
def execute_query(connection, query):
    '''
    This function will excecute a query passed into it 
    '''
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

#Upload a file into a db
def execute_many_query(connection, query, file):
    '''
    When inserting multiple values in form of rows in a db
    '''
    cursor = connection.cursor()
    rows = csv.reader(open(file),delimiter=';')
    try:
        cursor.executemany(query,rows)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

#read query
def execute_read_query(connection, query):
    '''
    Read the table function with a query
    '''
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")



In [3]:
connection = create_connection("E:\\sm_app.sqlite")

connection to SQLite DB succesful


In [4]:
##create prices table query
create_sales_table = """
CREATE TABLE IF NOT EXISTS sales (
  InvoiceNo INT,
  StockCode NONE,
  Description TEXT,
  Quantity INT, 
  InvoiceDate NUMERIC, 
  Price REAL,
  CustomerID INT,
  Country TEXT
) 
"""
execute_query(connection, create_sales_table)

Query executed successfully


## Load the Raw data to the DB 

In [5]:
execute_many_query(connection,"INSERT INTO sales (InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country) VALUES(?,?,?,?,?,?,?,?)", "online_retail.csv")

Query executed successfully


## Extraction and Segmentation

In [6]:
# The query for cohorts 
cohort_query = """
SELECT CustomerID,max(Recency) Recency,max(Frequency) Frequency,max(Monetary) Monetary from 
(SELECT CustomerID, ntile(4) over (ORDER BY max(InvoiceDate) asc) Recency,null Frequency ,null Monetary FROM sales group by CustomerID
UNION ALL 
SELECT CustomerID, null,ntile(4) over (ORDER BY count(InvoiceDate) asc) Frequency,null Monetary FROM sales
GROUP BY CustomerID
UNION ALL
SELECT CustomerID, null,null,ntile(4) over (ORDER BY sum(Quantity*Price) asc) Monetary FROM sales
GROUP BY CustomerID
)GROUP BY CustomerID
ORDER BY Recency,Frequency,Monetary
"""



In [7]:
def analyse_segment(query):
    '''
    This func Extracts the data and Segments them'''
    #Extract the data formt he Db 
    extract_= execute_read_query(connection, query)
    ext_list=[]
    for record in extract_:
         ext_list.append(record)
    df =pd.DataFrame(ext_list)
    
    # Change the column names
    df.set_axis(['CustomerId', 'Recency', 'Frequency', 'Monetary'], axis=1, inplace=True)
    df['Score']=df.iloc[:,1:4].sum(axis=1)
    
    # Group the scores into Loyalty levels
    loyalty = ['Bronze', 'Silver', 'Gold', 'Platinum']
    df['Loyalty_Level'] = pd.qcut(df['Score'], q=4, labels= loyalty)
    df.sort_values(by='Score', ascending=False,inplace=True)
    return df

In [8]:
analyse_segment(cohort_query)

,CustomerId,Recency,Frequency,Monetary,Score,Loyalty_Level
4373,,4,4,4,12,Platinum
4088,14191,4,4,4,12,Platinum
4077,14099,4,4,4,12,Platinum
4078,14110,4,4,4,12,Platinum
4079,14112,4,4,4,12,Platinum
...,...,...,...,...,...,...
219,16405,1,1,1,3,Bronze
220,16424,1,1,1,3,Bronze
221,16432,1,1,1,3,Bronze
222,16450,1,1,1,3,Bronze


In [9]:
#select_users = "DROP TABLE sales"
#select_users = "DELETE FROM trivcase1"
#execute_read_query(connection, select_users)